In [1]:
from __future__ import annotations
from dataclasses import dataclass, field
from typing import List, Optional

In [2]:
from cimgraph.databases import ConnectionParameters, RDFlibConnection
from cimgraph.models import FeederModel
import cimgraph.utils as utils
import importlib
from cimgraph import FeederModel

In [3]:
import easycim as easycim

In [4]:
cim_profile = 'cimhub_2023'
cim = importlib.import_module('cimgraph.data_profile.' + cim_profile)

# RDFLib File Reader Connection
params = ConnectionParameters(filename="/home/ande188/CIM-Graph/tests/test_models/ieee13.xml", cim_profile='cimhub_2023', iec61970_301=8)
# params = ConnectionParameters(filename="IEEE13_Assets.xml", cim_profile='cimhub_2023', iec61970_301=8)
rdf = RDFlibConnection(params)

In [5]:
feeder_mrid = "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62"
# feeder_mrid = "5B816B93-7A5F-B64C-8460-47C17D6E4B0F"
feeder = cim.Feeder(mRID = feeder_mrid)
network = FeederModel(connection=rdf, container=feeder, distributed=False)

In [6]:
network.pprint(cim.OverheadWireInfo)

{}


In [7]:
line_data = easycim.get_impedance_data_per_line(network)
geometry_data = easycim.get_geometry_data_per_line(network)
all_line_data = easycim.get_all_data(network, 'ACLineSegment')
geo_data2 = easycim.get_line_data_per_geometry(network)

load_data = easycim.get_load_data(network)
inverter_data = easycim.get_inverter_data(network)
swing_data = easycim.get_swing_bus_data(network)
xfmr_3ph = easycim.get_three_phase_transformer_data(network)

In [8]:
line_data.update(geometry_data)

In [9]:
import json
print(json.dumps(xfmr_3ph, indent=4))

{
    "259E820F-B4AF-4E1A-8271-687534EDAECC": {
        "vectorGroup": "Dyn1y1",
        "PowerTransformerEnd": [
            {
                "Terminal": {
                    "sequenceNumber": "2",
                    "ConnectivityNode": "650"
                },
                "endNumber": "2",
                "grounded": "true",
                "rground": "0",
                "xground": "0.4",
                "connectionKind": "Yn",
                "phaseAngleClock": "1",
                "ratedS": "5000000",
                "ratedU": "4160",
                "r": "0.0000173056",
                "StarImpedance": {},
                "ToMeshImpedance": [
                    {
                        "r": "0.02645",
                        "x": "0.2645",
                        "r0": "0.02645",
                        "x0": "0.2645"
                    }
                ],
                "FromMeshImpedance": [
                    {
                        "r": "0.0000346112",
        

In [10]:
network.get_all_edges(cim.EnergySource)
network.pprint(cim.EnergySource)

{
    "8B452737-CA3D-4C10-A99D-6865506437C8": {
        "mRID": "8B452737-CA3D-4C10-A99D-6865506437C8",
        "name": "source",
        "Location": "3818472D-6569-4901-849F-D5A68F6CDC24",
        "EquipmentContainer": "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62",
        "BaseVoltage": "A7580663-0A6B-43E7-B1DF-45A4D25F0F97",
        "Terminals": "['F9439A87-D8E9-406F-ADD3-336B29B8BCEA']",
        "nominalVoltage": "115000",
        "r": "0.16037668",
        "r0": "0.17960358",
        "voltageAngle": "0.52359878",
        "voltageMagnitude": "115000",
        "x": "0.64150673",
        "x0": "0.53881075"
    }
}


In [11]:
swing_data

{'8B452737-CA3D-4C10-A99D-6865506437C8': {'r': '0.16037668',
  'r0': '0.17960358',
  'x': '0.64150673',
  'x0': '0.53881075',
  'nominalVoltage': '115000',
  'voltageMagnitude': '115000',
  'voltageAngle': '0.52359878',
  'phases': []}}

In [12]:
@dataclass
class cimplified_ACLineSegment():
    mRID: Optional[str] = field(default=None)
    name: Optional[str] = field(default=None)
    r: Optional[str] = field(default=None)
    x: Optional[str] = field(default=None)
    bch: Optional[str] = field(default=None)
    r0: Optional[str] = field(default=None)
    x0: Optional[str] = field(default=None)
    b0ch: Optional[str] = field(default=None)
    phases: Optional[list] = field(default_factory=list)

    
@dataclass 
class cimplified_ACLineSegmentPhase():
    name: Optional[str] = field(default=None)
    phase: Optional[str] = field(default=None)
    



In [13]:
line = cimplified_ACLineSegment()
p1 = cimplified_ACLineSegmentPhase(name = '1')
p2 = cimplified_ACLineSegmentPhase(name = '2')

line.phases.append(p1)
line.phases.append(p2)
line.__dict__

{'mRID': None,
 'name': None,
 'r': None,
 'x': None,
 'bch': None,
 'r0': None,
 'x0': None,
 'b0ch': None,
 'phases': [cimplified_ACLineSegmentPhase(name='1', phase=None),
  cimplified_ACLineSegmentPhase(name='2', phase=None)]}

In [14]:
cim.ACLineSegment.__annotations__.keys()

dict_keys(['b0ch', 'bch', 'g0ch', 'gch', 'r', 'r0', 'shortCircuitEndTemperature', 'x', 'x0', 'ACLineSegmentPhases', 'Clamp', 'Cut', 'LineFaults', 'ParallelLineSegment', 'PerLengthImpedance', 'WireSpacingInfo'])

In [15]:
cim_class = cim.ACDCTerminal
print(json.dumps(list(cim_class.__annotations__.keys()), indent=12))


[
            "connected",
            "sequenceNumber",
            "BusNameMarker",
            "Measurements",
            "OperationalLimitSet"
]


In [16]:
network.get_all_edges(cim.Terminal)
network.__parsed__ = {}

In [17]:
network.get_all_edges(cim.PowerTransformer)
network.get_all_edges(cim.PowerTransformerEnd)
network.pprint(cim.PowerTransformerEnd)

{
    "FA1CDF26-3D23-45AE-AF62-5B924E86573A": {
        "mRID": "FA1CDF26-3D23-45AE-AF62-5B924E86573A",
        "name": "xfm1_End_2",
        "endNumber": "2",
        "grounded": "true",
        "rground": "0",
        "xground": "0",
        "BaseVoltage": "11A577F8-F9E2-41D1-81E3-6CF46A310F5C",
        "Terminal": "E1763FE3-82C2-4434-9424-72DF850D982E",
        "ToMeshImpedance": "['A9A75635-5FA4-434E-933E-27D9AE4AA462']",
        "connectionKind": "WindingConnection.Y",
        "phaseAngleClock": "0",
        "r": "0.0025344",
        "ratedS": "500000",
        "ratedU": "480",
        "PowerTransformer": "1E6B5C97-C4E8-4CED-B9A5-6E69F389DA93"
    },
    "B0352ED5-349E-4511-9663-EC1131757376": {
        "mRID": "B0352ED5-349E-4511-9663-EC1131757376",
        "name": "xfm1_End_1",
        "endNumber": "1",
        "grounded": "true",
        "rground": "0",
        "xground": "0",
        "BaseVoltage": "2A158E0C-CD01-4A50-AEBA-59D761FCF15D",
        "CoreAdmittance": "DE26A1C8-B45

In [18]:
l = []
l.append(cim.ACLineSegment)

cim.ACLineSegment in l

True

In [19]:
from typing import ClassVar